<h2>MAGMA trial</h2>
This notebook was created for trial purposes. MAGMA is a stand alone tool for gene-set analyis of GWAS data:<br>

 de Leeuw C, Mooij J, Heskes T, Posthuma D (2015): <strong> MAGMA: Generalized gene-set analysis of GWAS data.</strong> PLoS Comput Biol 11(4): e1004219. doi:10.1371/journal.pcbi.1004219)(<a href="https://journals.plos.org/ploscompbiol/article?id=10.1371%2Fjournal.pcbi.1004219" >link</a>).<br> 

 Tool documentation can be found <a href="https://vu.data.surfsara.nl/index.php/s/MUiv3y1SFRePnyG">here</a>

In [8]:
# Imports
import subprocess as sp
import pandas as pd
from pathlib import Path
from datetime import datetime
import sys
sys.path.append("../Src")
from MAGMA import run_gene_annotation, run_window_analysis, run_gene_analysis
from file_iterator import iterate_files

# Making sure MAGMA works
result = sp.run(f"magma --version", shell=True, capture_output=True, text=True)
print(result.stdout) if result.stdout else None
print(result.stderr) if result.stderr else None

MAGMA version: v1.10 (win/s)



In [ ]:
# Paths
DATA_DIR = Path("../Data/MAGMA")
SCDRS_DIR = Path("../Data/SCDRS")

# Reference file used for p-value geen analyis. Should point to the directory containing, along with the prefix for the .bed, .bim and .fam files. 
REFERENCE = Path("../Data/Reference/g1000_eur/g1000_eur")

In [ ]:
# Define traits for analysis. These should be the prefixes of the files in the DATA_DIR directory.
TRAITS = [
    # "SLE", 
    # "T1D",
    "Height",
]    
# Flag indicating to use open chromatine data
FILTERED = True

The gene location file must contain at least four columns, in this order: (1) **gene ID**, (2) **chromosome**, (3) **start site**, (4) **stop site**.<br> 
Optionally the strand of the gene can be included in a fifth column as well, though this is only used when annotating with an asymmetrical gene window.

<H3> <b>1. Gene annotation</b></H3>

The annotation step produces an output file with the .genes.annot suffix, with each row
corresponding to a gene, containing the **gene ID**, a specification of the **gene’s location**, and a list of **SNP
IDs of SNPs mapped to that gene**. To include a windows use the --anntotate flag;<br>  ‘--annotate window=5,1.5’ would set a 5kb upstream and 1.5kb downstream window.

<H4><i> 1.1 Gene window analysis </i></H4>
This script performs gene annotation window analysis by mapping SNPs to genes using adjustable window sizes. It iterates through SNP location files for specific traits, runs the MAGMA tool to perform the annotation for each window size, and collects results on the number of mapped SNPs and genes. The results are stored in a DataFrame and displayed for each trait. The window sizes (e.g., "0,0", "2,0.5", "20,5", "200,50") are customizable, allowing for flexible analysis of SNP-gene associations. Temporary files are deleted after the analysis.

In [ ]:
geneloc_file = DATA_DIR / "GENELOC.tsv"
window_sizes = ["0,0", "2,0.5", "20,5", "200,50"]

for snploc_file, trait in iterate_files(DATA_DIR / "snp_locations", TRAITS, FILTERED):
    annotation_results = run_window_analysis(
        snploc_file=snploc_file,
        geneloc_file=geneloc_file,
        window_sizes=window_sizes
    )
    display(annotation_results.style.set_caption(f"Annotation results for {trait}"))
    print("=" * 55)


,Window size,Mapped SNPs,Mapped genes
0,"0,0",591 (65.38%),288 genes (out of 33461)
1,"2,0.5",641 (70.91%),323 genes (out of 33461)
2,"20,5",769 (85.07%),518 genes (out of 33461)
3,"200,50",897 (99.23%),1749 genes (out of 33461)


,Window size,Mapped SNPs,Mapped genes
0,"0,0",205 (64.67%),113 genes (out of 33461)
1,"2,0.5",243 (76.66%),125 genes (out of 33461)
2,"20,5",278 (87.7%),202 genes (out of 33461)
3,"200,50",314 (99.05%),768 genes (out of 33461)


<h4><i>1.2 Writing genes.anno files</h4></i>

In [12]:
window = "20,5"
snp_dir = DATA_DIR / "snp_locations" / "open_chromatine" if FILTERED else DATA_DIR / "snp_locations" 

for snploc_file, trait in iterate_files(snp_dir, TRAITS, FILTERED):
    out_file = DATA_DIR / "gene_annotations" / trait

    print(f"Processing trait: {trait}...")
    result = run_gene_annotation(window=window, 
                                snploc_file=snploc_file,
                                geneloc_file=DATA_DIR / "GENELOC.tsv", 
                                out_file=out_file)
    print(result)
    print("=" * 90)
print("Done")

Processing trait: SLE...
Welcome to MAGMA v1.10 (win/s)
Using flags:
	--annotate window=20,5
	--snp-loc ..\Data\MAGMA\snp_locations\SLE_SNPLOC.tsv
	--gene-loc ..\Data\MAGMA\GENELOC.tsv
	--out ..\Data\MAGMA\gene_annotations\SLE

Start time is 11:51:11, Saturday 22 Feb 2025

Starting annotation...
Reading gene locations from file ..\Data\MAGMA\GENELOC.tsv... 
	adding window: 20000bp (before), 5000bp (after)
	33461 gene locations read from file
	chromosome  1: 3150 genes
	chromosome  2: 2281 genes
	chromosome  3: 1709 genes
	chromosome  4: 1353 genes
	chromosome  5: 1659 genes
	chromosome  6: 1630 genes
	chromosome  7: 1535 genes
	chromosome  8: 1318 genes
	chromosome  9: 1226 genes
	chromosome 10: 1228 genes
	chromosome 11: 1919 genes
	chromosome 12: 1761 genes
	chromosome 13: 666 genes
	chromosome 14: 1385 genes
	chromosome 15: 1133 genes
	chromosome 16: 1529 genes
	chromosome 17: 1894 genes
	chromosome 18: 662 genes
	chromosome 19: 2004 genes
	chromosome 20: 895 genes
	chromosome 21: 5

<H4><i>1.3 Gene analysis on SNP p-value data</i></h4>
Requires a binary PLINK format data set, consisting of a .bed, .bim and .fam trio of files. <br>
genotype data specified by --bfile is used to specify the reference data used to estimate LD between SNPs; <br> 
any phenotype contained in that data is ignored. (reference files can be downloaded <a href="https://cncr.nl/research/magma/">here</a>) <br>

Only the file prefix needs to be specified for the --bfile flag.

For gene analysis on raw genotype data the commmand: <b>magma --bfile [DATA] --gene-annot [ANNOT].genes.annot --out [OUTPUT_PREFIX]</b> can be used. <br>
In this case --bfile refers to the genotype data. 



In [18]:

run_dir = DATA_DIR / "gene_analysis" / datetime.now().strftime("%Y-%m-%d_%H-%M")
run_dir.mkdir(exist_ok=True)

for pval_file, trait in iterate_files(DATA_DIR / "pvals", TRAITS, FILTERED):
    gene_anno_file = DATA_DIR / "gene_annotations" / f"{trait}.genes.annot.txt"
    out_file = run_dir / trait

    print(f"Processing trait: {trait}...")
    run_gene_analysis(pval_file=pval_file, out_file=out_file, reference=REFERENCE, gene_anno_file=gene_anno_file, trait=trait)

Processing trait: T1D_OC...
T1D_OC  Welcome to MAGMA v1.10 (win/s)
T1D_OC  Using flags:
T1D_OC  	--bfile ..\Data\Reference\g1000_eur\g1000_eur
T1D_OC  	--gene-annot ..\Data\MAGMA\gene_annotations\T1D_OC.genes.annot.txt
T1D_OC  	--pval ..\Data\MAGMA\pvals\T1D_PVAL.txt use=1,2 ncol=TOTAL_SAMPLES
T1D_OC  	--out ..\Data\MAGMA\gene_analysis\2025-02-22_14-22\T1D_OC
T1D_OC  
T1D_OC  Start time is 14:22:46, Saturday 22 Feb 2025
T1D_OC  
T1D_OC  Loading PLINK-format data...
T1D_OC  Reading file ..\Data\Reference\g1000_eur\g1000_eur.fam... 503 individuals read
T1D_OC  Reading file ..\Data\Reference\g1000_eur\g1000_eur.bim... 22665064 SNPs read
T1D_OC  Preparing file ..\Data\Reference\g1000_eur\g1000_eur.bed... 
T1D_OC  
T1D_OC  Reading SNP synonyms from file ..\Data\Reference\g1000_eur\g1000_eur.synonyms (auto-detected)
T1D_OC  	read 6016767 mapped synonyms from file, mapping to 3921040 SNPs in the data
T1D_OC  	WARNING: detected 133 synonymous SNP pairs in the data
T1D_OC  	         skipped all

NOTE: Running the command in the shell directly is slightly faster. For very large data files this might save a decent amount of time. <br>
Moreover, only 14%(1 core likely) of my cpu and ~160-450mb of memory gets used(either in shell or in python), so running multiple instances of the command in parallel <br>
or looking into batch options might be a good idea.<br>
T1D 75 genes 12.53 py - 12.33 sh<br>
SLE 252 genes 54:50  py - 52:52 sh

<h4><i> 1.4 Formatting and writing genset file </h4></i>

In [ ]:
# TODO: Maybe look at file formatting for _OC, maintaining the _{suffix} might be nice for other modes though.
# Write scDRS gene set file
results_dir = run_dir
valid_traits = []
gene_set_data = {}

# Read Gene Analysis results and store gene scores
for file in results_dir.iterdir():
    if file.is_file() and file.name.endswith("genes.out.txt"):
        
        trait = file.name.split(".")[0]
        if trait in TRAITS or [trait.split("_")[0] for trait in TRAITS]:
            valid_traits.append(trait)
            
            with open(file, "r") as f:
                f.readline() # skip header
                for line in f:
                    line = line.split()
                    gene_score = f"{line[0]}:{line[7]}" # line[0] is gene name, line[7] is z-score
                    
                    if trait not in gene_set_data:
                        gene_set_data[trait] = gene_score 
                    else:
                        gene_set_data[trait] += "," + gene_score

gene_set_file = SCDRS_DIR / "genesets" / f"GS_{'-'.join(valid_traits)}_W{window}.gs"
pd.DataFrame(list(gene_set_data.items()), columns=["TRAIT", "GENESET"]).to_csv(gene_set_file, sep="\t", index=False)
print(f"Geneset written to {gene_set_file}")

Geneset written to ..\Data\SCDRS\genesets\GS_T1D_OC_W20,5.gs
